In [138]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [139]:
import numpy as np
from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

In [140]:
with open('/content/drive/MyDrive/Анна Каренина (отрывок).TXT', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '') 

In [141]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

In [142]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('лев', 1), ('николаевич', 1), ('толстой', 1), ('анна', 4), ('каренина', 2), ('мне', 11), ('отмщение', 1), ('и', 387), ('аз', 1), ('воздам', 1)]


In [143]:
data = tokenizer.texts_to_sequences([texts])
res = to_categorical(data[0], num_classes=maxWordsCount)
print(res.shape)

(6196, 1000)


In [144]:
inp_words = 3
n = res.shape[0] - inp_words

In [145]:
X = np.array([res[i:i + inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [146]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 128)               144512    
                                                                 
 dense_4 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 273,512
Trainable params: 273,512
Non-trainable params: 0
_________________________________________________________________


In [147]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [148]:
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
194/194 [==============================] - 3s 9ms/step - loss: 6.2384 - accuracy: 0.0578
Epoch 2/50
194/194 [==============================] - 2s 9ms/step - loss: 5.7942 - accuracy: 0.0625
Epoch 3/50
194/194 [==============================] - 2s 9ms/step - loss: 5.6410 - accuracy: 0.0706
Epoch 4/50
194/194 [==============================] - 2s 9ms/step - loss: 5.3929 - accuracy: 0.0943
Epoch 5/50
194/194 [==============================] - 2s 9ms/step - loss: 5.0241 - accuracy: 0.1277
Epoch 6/50
194/194 [==============================] - 2s 9ms/step - loss: 4.5904 - accuracy: 0.1744
Epoch 7/50
194/194 [==============================] - 2s 9ms/step - loss: 4.1456 - accuracy: 0.2117
Epoch 8/50
194/194 [==============================] - 2s 9ms/step - loss: 3.7381 - accuracy: 0.2567
Epoch 9/50
194/194 [==============================] - 2s 9ms/step - loss: 3.3738 - accuracy: 0.3007
Epoch 10/50
194/194 [==============================] - 2s 9ms/step - loss: 3.0605 - accuracy: 0.3391

In [149]:
def buildPhrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = to_categorical(data[i: i + inp_words], num_classes=maxWordsCount)  # преобразуем в One-Hot-encoding
        inp = x.reshape(1, inp_words, maxWordsCount)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res

In [152]:
res = buildPhrase("каждая несчастливая семья")
print(res)

1/1 [==============================] - 0s 19ms/step
каждая несчастливая семья несчастлива по своему все не ее и дал ей к и как и даже и своим и написал ей к
